In [6]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [2]:
df=pd.read_csv('train.csv')

In [7]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
99,100,0,2,"Kantor, Mr. Sinai",male,34.0,1,0,244367,26.0000,NaN,S
467,468,0,1,"Smart, Mr. John Montgomery",male,56.0,0,0,113792,26.5500,NaN,S
459,460,0,3,"O'Connor, Mr. Maurice",male,NaN,0,0,371060,7.7500,NaN,Q
568,569,0,3,"Doharr, Mr. Tannous",male,NaN,0,0,2686,7.2292,NaN,C
681,682,1,1,"Hassab, Mr. Hammad",male,27.0,0,0,PC 17572,76.7292,D49,C


In [9]:
df.drop(columns=['PassengerId', 'Cabin', 'Name', 'Ticket'], inplace=True)

In [10]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [11]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test=train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2)

In [17]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
310,1,female,24.0,0,0,83.1583,C
662,1,male,47.0,0,0,25.5875,S
222,3,male,51.0,0,0,8.0500,S
309,1,female,30.0,0,0,56.9292,C
247,2,female,24.0,0,2,14.5000,S


In [18]:
y_train.sample(5)

,Survived
358,1
856,1
785,0
560,0
308,0


In [20]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
310,1,female,24.0,0,0,83.1583,C
662,1,male,47.0,0,0,25.5875,S
222,3,male,51.0,0,0,8.0500,S
309,1,female,30.0,0,0,56.9292,C
247,2,female,24.0,0,2,14.5000,S


In [21]:
# Simple Imputer -Transformer 1
trf1= ColumnTransformer([
    ('impute_age', SimpleImputer(strategy='mean'), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])],
                        remainder='passthrough'

)

In [24]:
# One Hot Encoder - Transformer 2

trf2= ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown ='ignore'), [1, 6])
], remainder='passthrough')

In [26]:
# Min Max Scaling - Transformer 3
trf3= ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10)
    )
])

In [51]:
# Feature Extraction - Transformer 4
trf4=ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

In [52]:
# Training the model
trf5=DecisionTreeClassifier()

Creating Pipeline

In [53]:
pipe=Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

In [54]:
# Train
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf5', DecisionTreeClassifier())])

In [65]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf5': DecisionTreeClassifier()}

In [38]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.54094406])

In [42]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [66]:
y_predict=pipe.predict(X_test)

In [67]:
y_predict

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0])

In [68]:
from sklearn.metrics import accuracy_score

In [69]:
a_s=accuracy_score(y_test, y_predict)

In [70]:
a_s

0.5921787709497207

In [64]:
pipe.named_steps['trf4'].transformers_[0][1]

MinMaxScaler()

In [71]:
# Cross Validation Score
from sklearn.model_selection import cross_val_score

In [72]:
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6475032010243278

In [73]:
# GridSearchCV
from sklearn.model_selection import GridSearchCV

In [74]:
params={
    'trf5_max_depth':[1,2,3,4,5, None]
}

In [76]:
grid =GridSearchCV(pipe, params, cv=5, scoring='accuracy')

In [79]:
#Exporting the pipeline

import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))